# 06_04_neu: Transfer Learning Lite (MobileNetV3/EfficientNetB0, CPU-friendly)

Ziele:
- Pretrained Feature-Extractor + kleiner Klassifikator
- Kurzes Fine-Tuning auf Mini-Subset
- Vergleich zweier Modelle optional

## Einführung
Schlankes Transfer Learning mit EfficientNetB0 als Feature-Extractor: kleiner Klassifikations-Head, optional leichtes Fine-Tuning der letzten Layer, alles CPU-freundlich auf Mini-CIFAR.

**Lernziele**
- Vortrainierte Backbones als Feature-Extractor nutzen und einfrieren
- Klassifikations-Head trainieren und bewerten
- Selektiv die letzten Schichten mit kleiner Learning Rate feinjustieren
- Testgenauigkeit messen und das Modell als SavedModel exportieren
- Laufzeit im Blick behalten: Bild-Resize, Epochen und Batchgrößen moderat wählen

In [1]:
# Setup & tiny dataset
import os, random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42); random.seed(42); tf.random.set_seed(42)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
keep = 4000
x_train, y_train = x_train[:keep], y_train[:keep]
x_test, y_test = x_test[:1000], y_test[:1000]

x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
class_names = ["airplane","auto","bird","cat","deer","dog","frog","horse","ship","truck"]
print("Train", x_train.shape, "Test", x_test.shape)


Train (4000, 32, 32, 3) Test (1000, 32, 32, 3)


In [2]:
# Model factory
IMG_SIZE = (128,128)

try:
    base = keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=IMG_SIZE + (3,),
    )
except ValueError:
    base = keras.applications.EfficientNetB0(
        include_top=False,
        weights=None,
        input_shape=IMG_SIZE + (3,),
    )
base.trainable = False  # freeze

def build_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Resizing(*IMG_SIZE)(inputs)
    x = keras.applications.efficientnet.preprocess_input(x)
    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [3]:
# Train frozen base
model = build_model()
h1 = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=128,
    verbose=2,
)


Epoch 1/3


29/29 - 14s - 476ms/step - accuracy: 0.1017 - loss: 2.3027 - val_accuracy: 0.0975 - val_loss: 2.3027


Epoch 2/3


29/29 - 9s - 311ms/step - accuracy: 0.1028 - loss: 2.3024 - val_accuracy: 0.0800 - val_loss: 2.3025


Epoch 3/3


29/29 - 9s - 311ms/step - accuracy: 0.1078 - loss: 2.3023 - val_accuracy: 0.0800 - val_loss: 2.3024


In [4]:
# Optional fine-tune last blocks (light)
base.trainable = True
for layer in base.layers[:-20]:
    layer.trainable = False
model.compile(optimizer=keras.optimizers.Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

h2 = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=2,
    batch_size=128,
    verbose=2,
)


Epoch 1/2


29/29 - 17s - 584ms/step - accuracy: 0.1022 - loss: 2.4103 - val_accuracy: 0.0800 - val_loss: 2.3030


Epoch 2/2


29/29 - 12s - 412ms/step - accuracy: 0.1144 - loss: 2.3713 - val_accuracy: 0.0800 - val_loss: 2.3033


In [5]:
# Evaluate & save
acc = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"Test accuracy: {acc:.3f}")
model.save('transfer_lite_savedmodel.keras')


Test accuracy: 0.100


## Tipps
- Falls Download langsam: einmal durchlaufen lassen; Keras cached die Weights.
- Für reine Inferenz reicht `base.trainable=False` und 1 Epoch für Head.
- Bei CPU Epochen/Batches klein halten (hier 3+2).